# Data Visualizer

## Rendering functions


In [45]:
import trimesh
import pyrender
import numpy as np
import imageio
from pathlib import Path
import os
from concurrent.futures import ThreadPoolExecutor

def transformation_matrix(roll = 0, pitch = 0, yaw = 0, x = 0, y = 0, z = 0):
        roll = np.radians(roll)
        pitch = np.radians(pitch)
        yaw = np.radians(yaw)
        # Compute rotation matrices
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(roll), -np.sin(roll)],
            [0, np.sin(roll), np.cos(roll)]
        ])
        
        Ry = np.array([
            [np.cos(pitch), 0, np.sin(pitch)],
            [0, 1, 0],
            [-np.sin(pitch), 0, np.cos(pitch)]
        ])
        
        Rz = np.array([
            [np.cos(yaw), -np.sin(yaw), 0],
            [np.sin(yaw), np.cos(yaw), 0],
            [0, 0, 1]
        ])
        
        # Combined rotation
        R = Rz @ Ry @ Rx  # Note: order matters!

        # Build the full transformation matrix
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = [x, y, z]
        
        return T

def render_obj(obj_path, distance = 1.5, bg_color = [1,1,1,0], object_color = [128,128,128,255],n_frames=36, image_size=(512, 512)):
    obj_path = Path(obj_path)
    mesh = trimesh.load(obj_path, force='mesh')
    mesh.visual = trimesh.visual.ColorVisuals(mesh)
    # Set all vertex colors to white (RGBA = [255, 255, 255, 255])
    mesh.visual.vertex_colors = np.array(object_color)



    # Set up scene
    scene = pyrender.Scene(bg_color = bg_color, ambient_light = [0.3, 0.3, 0.3])  # White background
    mesh_pyrender = pyrender.Mesh.from_trimesh(mesh, smooth=True)
    scene.add(mesh_pyrender)

    # Camera settings
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    cam_node = scene.add(camera, pose=np.eye(4))

    # Light settings
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=2.0)
    scene.add(light, pose=np.eye(4))
    light2 = pyrender.DirectionalLight(color=np.ones(3), intensity=1.0)
    scene.add(light2, pose=transformation_matrix(pitch = 180))


    # Renderer
    r = pyrender.OffscreenRenderer(viewport_width=image_size[0], viewport_height=image_size[1])

    

    # Standard starting camera pose
    def create_camera_pose(angle_deg, distance=1.5):
        angle_rad = np.radians(angle_deg)
        eye = np.array([np.sin(angle_rad) * distance, 0.7 * distance, np.cos(angle_rad) * distance])
        center = np.array([0.0, 0.0, 0.0])
        up = np.array([0.0, 1.0, 0.0])

        z = (eye - center)
        z /= np.linalg.norm(z)
        x = np.cross(up, z)
        x /= np.linalg.norm(x)
        y = np.cross(z, x)
        y /= np.linalg.norm(y)

        pose = np.eye(4)
        pose[0:3, 0] = x
        pose[0:3, 1] = y
        pose[0:3, 2] = z
        pose[0:3, 3] = eye
        return pose

    # Save single image (first frame)
    cam_pose = create_camera_pose(0, distance=distance)
    scene.set_pose(cam_node, pose=cam_pose)
    color, _ = r.render(scene)
    image_path = obj_path.with_suffix('.png')
    imageio.imwrite(image_path, color)

    # Generate frames for GIF
    frames = []
    for i in range(n_frames):
        angle = (360 / n_frames) * i
        cam_pose = create_camera_pose(angle)
        scene.set_pose(cam_node, pose=cam_pose)
        color, _ = r.render(scene)
        frames.append(color)

    # Save GIF
    gif_path = obj_path.with_suffix('.gif')
    imageio.mimsave(gif_path, frames, fps=12)

    r.delete()
    print(f"Saved {image_path} and {gif_path}")

def process_all_objs(folder_path, distance = 1.5, bg_color = [1,1,1,0], object_color = [128,128,128,255],n_frames=36, image_size=(512, 512)):
    folder_path = Path(folder_path)
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".obj"):
                render_obj(os.path.join(root, file), distance = distance, bg_color = bg_color, object_color = object_color,n_frames=n_frames, image_size=image_size) 




In [ ]:
# zoekt alle .obj's in een map en maakt er een screenshot van en een gifje in dezelfde map.
process_all_objs(r"C:\Users\jelle\Documents\DoctoraatLocal\completiontools-evaluation\data\results",
                 distance = 1.5, # the distance away from the object
                 bg_color = [1.0, 1.0, 1.0, 0.0], #white (1.0 max)
                 object_color = [128,128,128,255], # middle gray (255 max)
                 n_frames=36, 
                 image_size=(512, 512)
                 )


Saved C:\Users\jelle\Documents\DoctoraatLocal\completiontools-evaluation\data\results\recht_bocht_180\gt.png and C:\Users\jelle\Documents\DoctoraatLocal\completiontools-evaluation\data\results\recht_bocht_180\gt.gif
Saved C:\Users\jelle\Documents\DoctoraatLocal\completiontools-evaluation\data\results\recht_bocht_180\sdf.png and C:\Users\jelle\Documents\DoctoraatLocal\completiontools-evaluation\data\results\recht_bocht_180\sdf.gif
